In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from PIL import Image

In [3]:
class ImageFileDataset(torch.utils.data.Dataset):
    def __init__(self, folder_path, transform=None, label=0):
        self.folder_path = folder_path
        self.file_list = [f for f in os.listdir(folder_path) if f.endswith('.png')]  # .tiff extension
        self.transform = transform
        self.label = label

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        filename = self.file_list[idx]
        file_path = os.path.join(self.folder_path, filename)
        img = Image.open(file_path)
        if self.transform:
            img = self.transform(img)
        return img, self.label

Define the source of data

In [16]:
transform = transforms.Compose([
    transforms.Resize((1024,1024)),  # Resize image to 64x64
    transforms.ToTensor()])     # Convert image to PyTorch tensor

In [17]:
current_dir = os.getcwd()
rbc_sma_relative_path='rbc_sma'
rbc_non_sma_relative_path = 'rbc_non_sma'
rbc_sma_dir = os.path.join(current_dir, rbc_sma_relative_path)
rbc_non_sma_dir = os.path.join(current_dir, rbc_non_sma_relative_path)

In [18]:
dataset_sma = ImageFileDataset(rbc_sma_dir, transform=transform, label=1)
dataset_non_sma = ImageFileDataset(rbc_non_sma_dir, transform=transform, label=0)

In [19]:
combined_dataset = torch.utils.data.ConcatDataset([dataset_sma, dataset_non_sma])

# Get the size of dataset
dataset_size = len(combined_dataset)
print('Dataset: ',dataset_size, ' images')

# Define the splits
train_size = int(0.7 * dataset_size)
validation_size = int(0.1 * dataset_size)
test_size = dataset_size - train_size - validation_size

# Split combined dataset
train_dataset, validation_dataset, test_dataset = random_split(combined_dataset, [train_size, validation_size, test_size])
train_len = len(train_dataset)
val_len = len(validation_dataset)
test_len = len(test_dataset)

print('Training dataset: ',train_len, ' images')
print('Validation dataset: ',val_len, ' images')
print('Test dataset: ',test_len, ' images')



Dataset:  353  images
Training dataset:  247  images
Validation dataset:  35  images
Test dataset:  71  images


In [20]:
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=1, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

In [21]:
# Big CNN
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 128 * 128, 256)  # Adjusted for 1024x1024 images
        self.fc2 = nn.Linear(256, 2)  # 2 output classes: SMA and non-SMA

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))  # Additional conv and pool layer
        x = x.view(-1, 128 * 128 * 128)  # flatten the tensor, adjusted for 1024x1024 images
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = CNN()

# Smaller CNN
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 256 * 256, 128)  # Adjusted for reduced complexity
        self.fc2 = nn.Linear(128, 2)  # 2 output classes: SMA and non-SMA

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 32 * 256 * 256)  # flatten the tensor, adjusted for reduced complexity
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model_simple = SimpleCNN()

In [22]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_simple.parameters())

In [23]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_simple = model_simple.to(device)

In [24]:
from tqdm import tqdm

# Training loop
num_epochs = 1  # Number of training epochs
for epoch in range(num_epochs):
    model_simple.train()  # Set the model to training mode
    loop = tqdm(enumerate(train_loader), total=len(train_loader), leave=True)
    for i, (images, labels) in loop:
        # Move images and labels to the device
        # print(images)

        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model_simple(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Update progress bar
        loop.set_description(f"Epoch [{epoch+1}/{num_epochs}]")
        loop.set_postfix(loss = loss.item())
        
# Validation loop
model_simple.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Disable gradient computation
    correct = 0
    total = 0
    loop = enumerate(validation_loader)
    # print(loop)
    for i, (images, labels) in loop:

        images = images.to(device)
        labels = labels.to(device)
        outputs = model_simple(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Validation accuracy: {100 * correct / total}%')

Epoch [1/1]: 100%|██████████| 247/247 [40:23<00:00,  9.81s/it, loss=0.526]


Validation accuracy: 60.0%


Test Error

In [25]:
# Test loop
model.eval()  # Set the model to evaluation mode
test_loss = 0
correct = 0
total = 0

with torch.no_grad():  # Disable gradient computation
    loop = tqdm(enumerate(test_loader), total=len(test_loader), leave=True)
    for i, (images, labels) in loop:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        print('DONE')
    print(f'Test loss: {test_loss / (total / test_loader.batch_size)}, Test accuracy: {100 * correct / total}%')



  0%|          | 0/71 [00:00<?, ?it/s]

  1%|▏         | 1/71 [00:11<13:49, 11.86s/it]

DONE


  3%|▎         | 2/71 [00:16<09:00,  7.84s/it]

DONE


...

DONE


 99%|█████████▊| 70/71 [07:00<00:06,  6.81s/it]

DONE


100%|██████████| 71/71 [07:03<00:00,  5.97s/it]

DONE
Test loss: 0.6917832779212737, Test accuracy: 52.11267605633803%
